In [176]:
import praw
from _keys import client_id, secret_key, username, user_agent, password, db_user, db_password, db_name, db_host, db_port
import psycopg2
import time

In [177]:
reddit = praw.Reddit(client_id=client_id, client_secret=secret_key, username=username, password=password, user_agent=user_agent)
conn = psycopg2.connect(dbname=db_name, user=db_user, password=db_password, host=db_host, port=db_port)
cur = conn.cursor()

In [178]:
sql = """select sus_user_name
from public.sus_users"""

In [179]:
# Get all sus_user_names and save to list
names = []
cur.execute(sql)
for row in cur:
    names.append(row[0])

In [180]:
# table_column_sql = """select column_name
# from information_schema.columns
# where table_schema = 'public'
# and table_name = 'sus_user_posts'"""

In [181]:
# table_columns = []
# data_inserts_columns = []
# api_calls_columns = []
# cur.execute(table_column_sql)
# for row in cur:
#     #add to lists and remove "'"
#     table_columns.append(row[0].strip("'"))
#     # table_columns.append(row[0])
#     data_inserts_columns.append('%s')
#     api_calls_columns.append(f'submission.{row[0]}')

In [182]:
sql_insert = """INSERT INTO public.sus_user_posts (author, author_fullname, author_flair_text, author_is_blocked, author_premium, can_mod_post,
 created_utm, domain, id, is_created_from_ads_ui, is_crosspostable, is_meta, is_original_content, is_reddit_media_domain, is_robot_indexable,
 is_self, is_video, locked, over_18, permalink, subreddit_id, title, url, post_score, subreddit, top_lvl_comments, total_comments) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"""

In [183]:
for name in names:
    try:
        user = reddit.redditor(name)
        submissions = user.submissions.top(limit=None)
        for submission in submissions:
            cur.execute(sql_insert, (name, submission.author_fullname, submission.author_flair_text, submission.author_is_blocked, submission.author_premium, submission.can_mod_post, submission.created_utc, submission.domain, submission.id, submission.is_created_from_ads_ui, submission.is_crosspostable, submission.is_meta, submission.is_original_content, submission.is_reddit_media_domain, submission.is_robot_indexable, submission.is_self, submission.is_video, submission.locked, submission.over_18, submission.permalink, submission.subreddit_id, submission.title, submission.url, submission.score, submission.subreddit.display_name, submission.comments.__len__(), len(submission.comments.list())))
            conn.commit()
    except Exception as e:
        print(e)
        time.sleep(1)
        continue